# Understanding FastAPI Middleware and ASGI Concepts

In [ ]:
# NOTE: This notebook explains FastAPI's middleware chain processing,
#   so even if that logic isn't currently mentioned in the codebase,
#   it could prove useful in the future if we need to add a custom middleware
#   that inherits from a middleware that we're currently using.

## ASGI Important Keywords (Optional Read)

Below is an example of the ASGI components' details when processing a request (just for you to visualize the flow):


##### 1. **Client Request**
   - Makes HTTP request with headers and body
   - Example: POST request to create user

##### 2. **scope** (I.e., The Request's "Info"/A.K.A "context"/A.K.A "scope")
   ```python
   scope = {
       'type': 'http',
       'method': 'POST',
       'path': '/api/users',
       'headers': [(b'content-type', b'application/json')]
   }
   ```

##### 3. **receive** (i.e., The Request's Body "Recieved" From the Client)
   ```python
   message = await receive()
   # Returns:
   {
       'type': 'http.request',
       'body': b'{"username":"john"}',
       'more_body': False
   }
   ```

##### 4. **send** (I.e., the backend server's response that's "Sent" back to client after processing request in a FastAPI endpoint) 
   ```python
   # 1. Start response
   await send({
       'type': 'http.response.start',
       'status': 200,
       'headers': [...]
   })
   
   # 2. Send body
   await send({
       'type': 'http.response.body',
       'body': b'{"status":"success"}'
   })
   ```

##### 5. All of the keywords above are used in **__call__()** method of FastAPI's Middleware Class
   ```python
   # NOTE: This method is called when a client request is sent to the server
   async def __call__(self, scope, receive, send):
       # 1. Gets request info/body from scope/receive
       # ... request_details ... 

       # 2. Process request
       # (This basically sends the client request's info/body to FastAPI's endpionts 
       # where the backend dev. implemented their code)
       # result = process_request(request_details)
       
       # 3. Send response
       # (uses the send() method of the FastAPI's middleware class)
       # await send(result)
   ```



## Mapping Above Logic to CustomCORSMiddleware

Here's a heavily commented version of the custom CORS middleware:

In [ ]:
from fastapi.middleware.cors import CORSMiddleware
from fastapi.requests import Request
from loguru import logger

origins = [
    "http://localhost:8080",
]


class CustomCORSMiddleware(CORSMiddleware):
    async def __call__(self, scope, receive, send):
        """Entry point for handling requests through this middleware.

        Args:
            scope: Dict containing request info (method, headers, etc.)
            receive: Async callable for getting request body chunks
            send: Async callable for sending response chunks
        """
        # Skip non-HTTP requests (e.g., WebSocket)
        if scope["type"] != "http":
            return await super().__call__(scope, receive, send)

        logger.debug("Starting CORS middleware processing")

        # Create FastAPI Request object for easier access to request data
        request = Request(scope, receive)

        # Define a custom send function to intercept responses
        async def modified_send(message):
            """Intercept response messages to add logging.

            Args:
                message: Dict containing response data
                Example: {'type': 'http.response.start', 'status': 200}
            """
            if message["type"] == "http.response.start":
                status = message["status"]

                # Check CORS-related errors
                if request.headers.get("origin") not in origins:
                    logger.error(
                        f"CORS Origin Error | Status: {status} | "
                        f"Origin: {request.headers.get('origin')} | "
                        f"Allowed: {origins}"
                    )

                # Check other error conditions
                elif any(
                    [
                        # CORS preflight error
                        status == 400 and request.method == "OPTIONS",
                        # Auth header missing
                        status == 401 and "Authorization" not in request.headers,
                        # CORS origin forbidden
                        status == 403,
                        # Rate limit exceeded
                        status == 429,
                    ]
                ):
                    logger.error(f"Middleware Error | Status: {status} | " f"Path: {request.url.path}")

            # Forward the message to the original send function
            await send(message)

        try:
            # Call parent CORSMiddleware with our modified send function
            await super().__call__(scope, receive, modified_send)
        except Exception as e:
            # Log any unhandled errors
            logger.error(f"Unhandled Middleware Error: {str(e)}")
            raise


## Request Flow Visualization

```
┌──────────┐         ┌─────────────┐         ┌────────────┐
│  Client  │  --->   │ Middleware  │  --->   │  Endpoint  │
└──────────┘         └─────────────┘         └────────────┘
                           |
                     ┌─────┴─────┐
                     │  __call__  │
                     └─────┬─────┘
                           |
              ┌────────────┴────────────┐
              │                         │
        ┌────────┐               ┌──────────┐
        │ scope  │               │ modified │
        │(request│               │   send   │
        │ info)  │               │(response │
        └────────┘               │intercept)│
                                 └──────────┘
```

This diagram shows how:
1. Client request arrives at middleware
2. `__call__` processes the request using `scope` data
3. Response is intercepted by `modified_send`
4. Logging occurs based on response status
5. Response continues to client

#### Key Points to Remember About Middleware Inheritance (Optional):

1. **Constructor Inheritance**:
   - CustomCORSMiddleware inherits all initialization logic
   - No need to reimplement CORS configuration handling

2. **__call__ Override**:
   - Only overrides the ASGI entry point
   - Maintains original CORS logic through `super().__call__`
   - Adds logging through `modified_send` wrapper

3. **Send Function Wrapping**:
   - Original `send` is wrapped by `modified_send`
   - Allows response inspection before forwarding
   - Maintains ASGI protocol compatibility

This pattern allows us to:
- Add functionality without duplicating CORS logic
- Maintain full compatibility with FastAPI's CORS handling
- Insert logging at precise points in the request/response cycle